In [ ]:
# Nov. 11, 2021, Xiaole Zhang
# Download meteo data from ecmwf 
import cdsapi
from datetime import date, timedelta
import os
from pathlib import Path
import calendar 

# covert number to formated string
def number2String(f):
    return '%02d' % f

c = cdsapi.Client() 

# [up, left, down, right]
regionName = 'Beijing'
regionBound = {'Beijing': [50, 105, 30, 127],\
               'Asia': [55, 60, 0, 150]}

# data for Beijing
targetYear = 2021

targetMonths = [3]
monthsList = list(map(number2String, targetMonths))

targetDays = range(1,32)
daysList = list(map(number2String, targetDays))

# to download the meteo day by month to meet the volume requirement
for currentMonth in targetMonths:
    currentDay = date(targetYear, currentMonth, 1) 
    initalDay = date(targetYear, currentMonth, 1) 
    
    year = '%4d' % currentDay.year
    month = '%02d' % currentDay.month
    day = '%02d' % currentDay.day
    print('Current Day:' + currentDay.isoformat())
    
    # download single level data
    fileName = regionName + '/'+'SingleLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    filePath = Path(regionName + '/')
    filePath.mkdir(parents=True, exist_ok=True)
    c.retrieve( 
        'reanalysis-era5-single-levels', 
        { 
            'product_type': 'reanalysis', 
            'format': 'grib', 
            'variable': [ 
                '2m_temperature', 'boundary_layer_height', 'eastward_turbulent_surface_stress', 
                'evaporation', 'high_cloud_cover', 'medium_cloud_cover', 
                'northward_turbulent_surface_stress', 'skin_temperature', 'surface_sensible_heat_flux', 
                'surface_solar_radiation_downwards', 'surface_pressure', 'convective_precipitation','large_scale_precipitation','volumetric_soil_water_layer_1', 
            ], 
            'year': year, 
            'month': month,
            'day': daysList, 
            'time': [ 
                '00:00', '01:00', '02:00', 
                '03:00', '04:00', '05:00', 
                '06:00', '07:00', '08:00', 
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ], 
            'area': regionBound[regionName], 
        }, 
        fileName) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file'
    os.system("grib_filter "+ruleFile+" "+fileName)
    
    ###############################################
    # download 3d data
    fileName3d =  regionName + '/'+'PressureLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    c.retrieve( 
        'reanalysis-era5-pressure-levels', 
        { 
        'product_type': 'reanalysis', 
        'variable': [ 
        'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity', 
        'temperature', 'u_component_of_wind', 'v_component_of_wind', 
        ], 
        'pressure_level': [ 
        '500', '550', '600', 
        '650', '700', '750', 
        '775', '800', '825', 
        '850', '875', '900', 
        '925', '950', '975', 
        '1000', 
        ], 
        'year': year, 
        'month': month,
        'day': daysList, 
        'time': [ 
        '00:00', '01:00', '02:00', 
        '03:00', '04:00', '05:00', 
        '06:00', '07:00', '08:00', 
        '09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00',
        '15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00',
        '21:00', '22:00', '23:00', 
        ], 
        'area': regionBound[regionName], 
        'format': 'grib', 
        }, 
        fileName3d) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file_pressure'
    os.system("grib_filter "+ruleFile+" "+fileName3d)
    
    ###########################
    # combine the daily 2d and 3d data files into single files and delete the splitted data
    dayRange = calendar.monthrange(targetYear, currentMonth)
    for day in range(dayRange[0], dayRange[1]):
        currentDay = initalDay + timedelta(days=day) 
        print('Current Day:' + currentDay.isoformat())
        fileName3d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'_pressureLevels.grib'
        fileName2d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'.grib'
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        os.system("cat "+fileName2d+" "+fileName3d+" > "+fileNameAll)
        os.system("rm "+fileName2d+" "+fileName3d)    
   #fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
   #os.system("cat "+fileName+" "+fileName3d+" > "+fileNameAll)
   #os.system("rm "+fileName+" "+fileName3d)

2021-11-11 19:54:24,038 INFO Welcome to the CDS
2021-11-11 19:54:24,038 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-11-11 19:54:24,088 INFO Request is queued


Current Day:2021-03-01


In [34]:
fileName

'Beijing/SingleLevel_20210101.grib'

In [27]:

def number2String(f):
    return '%02d' % f
  
# We double all numbers using map()
numbers = range(1,32)
days = map(number2String, numbers)

In [ ]:
import cdsapi
from datetime import date, timedelta
import os
from pathlib import Path

c = cdsapi.Client() 

# [up, left, down, right]
regionName = 'Beijing'
regionBound = {'Beijing': [50, 105, 30, 127],\
               'Asia': [55, 60, 0, 150]}

# data for Beijing
startDay = date(2021, 2, 1)
durationDays = 45

def number2String(f):
    return '%02d' % f
  
# We double all numbers using map()
numbers = range(1,32)
dayStrings = map(number2String, numbers)


for daysN in range(0,durationDays):
    durations = timedelta(days=daysN)
    currentDay = durations+startDay
    year = '%4d' % currentDay.year
    month = '%02d' % currentDay.month
    day = '%02d' % currentDay.day
    print('Current Day:' + currentDay.isoformat())
      
    fileName3d =  regionName + '/'+'PressureLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    c.retrieve( 
        'reanalysis-era5-pressure-levels', 
        { 
            'product_type': 'reanalysis', 
            'variable': [ 
                'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity', 
                'temperature', 'u_component_of_wind', 'v_component_of_wind', 
            ], 
            'pressure_level': [ 
                '500', '550', '600', 
                '650', '700', '750', 
                '775', '800', '825', 
                '850', '875', '900', 
                '925', '950', '975', 
                '1000', 
            ], 
            'year': year, 
            'month':  [
                '02',
                ], 
            'day': list(dayStrings),
            'time': [ 
                '00:00', '01:00', '02:00', 
                '03:00', '04:00', '05:00', 
                '06:00', '07:00', '08:00', 
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00', 
            ], 
            'area': regionBound[regionName], 
            'format': 'grib', 
        }, 
        fileName3d) 

2021-11-11 17:04:45,389 INFO Welcome to the CDS
2021-11-11 17:04:45,389 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2021-11-11 17:04:45,424 INFO Request is queued


Current Day:2021-02-01


2021-11-11 18:33:11,903 INFO Request is running
2021-11-11 18:55:14,210 INFO Request is completed
2021-11-11 18:55:14,211 INFO Downloading https://download-0009.copernicus-climate.eu/cache-compute-0009/cache/data8/adaptor.mars.internal-1636651989.416678-12256-5-ca7be7c2-93cd-4af0-bb44-1a5199d64224.grib to Beijing/PressureLevel_20210201.grib (826.9M)
2021-11-11 18:56:43,116 INFO Download rate 9.3M/s  
2021-11-11 18:56:43,264 INFO Welcome to the CDS
2021-11-11 18:56:43,265 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2021-11-11 18:56:43,331 INFO Request is queued


Current Day:2021-02-02


In [25]:
# Nov. 11, 2021, Xiaole Zhang
# Download meteo data from ecmwf 
import cdsapi
from datetime import date, timedelta
import os
from pathlib import Path
import calendar 

# covert number to formated string
def number2String(f):
    return '%02d' % f

c = cdsapi.Client() 

# [up, left, down, right]
regionName = 'Beijing'
regionBound = {'Beijing': [50, 105, 30, 127],\
               'Asia': [55, 60, 0, 150]}

# data for Beijing
targetYear = 2021

targetMonths = [2]
monthsList = list(map(number2String, targetMonths))

targetDays = range(1,32)
daysList = list(map(number2String, targetDays))

# to download the meteo day by month to meet the volume requirement
for currentMonth in targetMonths:
    currentDay = date(targetYear, currentMonth, 1) 
    initalDay = date(targetYear, currentMonth, 1) 
    
    year = '%4d' % currentDay.year
    month = '%02d' % currentDay.month
    day = '%02d' % currentDay.day
    print('Current Day:' + currentDay.isoformat())
    fileName3d =  regionName + '/'+'PressureLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    print(fileName3d)
    
    ruleFile =regionName+'/rules_file_pressure'
    os.system("grib_filter "+ruleFile+" "+fileName3d)
    
    ruleFile =regionName+'/rules_file'
    fileName2d = regionName + '/'+'SingleLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    os.system("grib_filter "+ruleFile+" "+fileName2d)
    
    dayRange = calendar.monthrange(targetYear, currentMonth)
    for day in range(dayRange[0], dayRange[1]):
        currentDay = initalDay + timedelta(days=day) 
        print('Current Day:' + currentDay.isoformat())
        fileName3d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'_pressureLevels.grib'
        fileName2d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'.grib'
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        os.system("cat "+fileName2d+" "+fileName3d+" > "+fileNameAll)
        os.system("rm "+fileName2d+" "+fileName3d)

Current Day:2021-02-01
Beijing/PressureLevel_20210201.grib
Current Day:2021-02-01
Current Day:2021-02-02
Current Day:2021-02-03
Current Day:2021-02-04
Current Day:2021-02-05
Current Day:2021-02-06
Current Day:2021-02-07
Current Day:2021-02-08
Current Day:2021-02-09
Current Day:2021-02-10
Current Day:2021-02-11
Current Day:2021-02-12
Current Day:2021-02-13
Current Day:2021-02-14
Current Day:2021-02-15
Current Day:2021-02-16
Current Day:2021-02-17
Current Day:2021-02-18
Current Day:2021-02-19
Current Day:2021-02-20
Current Day:2021-02-21
Current Day:2021-02-22
Current Day:2021-02-23
Current Day:2021-02-24
Current Day:2021-02-25
Current Day:2021-02-26
Current Day:2021-02-27
Current Day:2021-02-28


In [21]:
tt = (calendar.monthrange(targetYear, currentMonth))
tt[0]

0